In [128]:
import requests
import lxml.html
import pandas as pd
import sqlite3
from pandas.io import sql
import os

In [129]:
# response = requests.get('https://gnews.gg.go.kr/briefing/brief_gongbo.do?page=1&BS_CODE=s017&period_1=2020-05-01&period_2=2020-05-31&search=1&keyword=&subject_Code=BO01')
# root = lxml.html.fromstring(response.content)
# for a in root.cssselect('a'):
#     url = a.get('href')
#     print(url,a.text)

In [130]:
# response = requests.get('https://gnews.gg.go.kr/briefing/brief_gongbo.do?page=45&BS_CODE=s017&period_1=&period_2=&search=0&keyword=&subject_Code=BO01')
# root = lxml.html.fromstring(response.content)
# for li in root.xpath('//*[@id="chk-table"]/tbody/tr/td[2]'):
#     for a in li.xpath('a'): #리스트로 가져오려는 속성이 있음
#         url = a.get('href')
#         print(a.text)

In [131]:
def db_save(GG_LIST):
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            GG_LIST.to_sql(name = 'GG_LIST', con = con, index = False, if_exists='append') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(GG_LIST), '건 저장완료..')

In [132]:
def db_select():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            query = 'SELECT * FROM GG_LIST'
            GG_LIST = pd.read_sql(query, con = con)
        except Exception as e:
            print(str(e)) 
        return GG_LIST   

In [138]:
def db_delete():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            cur = con.cursor()
            sql = 'DELETE FROM GG_LIST'
            cur.execute(sql)
        except Exception as e:
            print(str(e)) 

In [158]:
db_delete()

In [159]:
page = 45
max_page = 75
#dt = '20200630'
while(True):
    df_list = []
    response = requests.get('https://gnews.gg.go.kr/briefing/brief_gongbo.do?page={}&BS_CODE=s017&period_1=&period_2=&search=0&keyword=&subject_Code=BO01'\
                            .format(page))
    root = lxml.html.fromstring(response.content)
    for tr in root.xpath('//*[@id="chk-table"]/tbody/tr'):
            a = tr.xpath('td[2]/a')[0]
            url = a.get('href')
            dept = tr.xpath('td[3]')[0]
            reg_date = tr.xpath('td[5]')[0]
            view = tr.xpath('td[6]')[0]
            
        #print(a.text)
        
            df = pd.DataFrame({
                 'url' : ['https://gnews.gg.go.kr'+url],
                 'title':[a.text],
                 'dept':[dept.text],
                 'reg_date':[reg_date.text.strip()],
                 'view':[view.text]
                              })
            df_list.append(df)

        
    df_10 = pd.concat(df_list)
    db_save(df_10)

#     if df_list:
#         df_10 = pd.concat(df_list) #인덱스 리셋
#         db_save(df_10)

    if  page >= max_page:
        break;
    else:
        page = page + 1


10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..
10 건 저장완료..


In [160]:
print(db_select())

                                                   url  \
0    https://gnews.gg.go.kr/briefing/brief_gongbo_v...   
1    https://gnews.gg.go.kr/briefing/brief_gongbo_v...   
2    https://gnews.gg.go.kr/briefing/brief_gongbo_v...   
3    https://gnews.gg.go.kr/briefing/brief_gongbo_v...   
4    https://gnews.gg.go.kr/briefing/brief_gongbo_v...   
..                                                 ...   
305  https://gnews.gg.go.kr/briefing/brief_gongbo_v...   
306  https://gnews.gg.go.kr/briefing/brief_gongbo_v...   
307  https://gnews.gg.go.kr/briefing/brief_gongbo_v...   
308  https://gnews.gg.go.kr/briefing/brief_gongbo_v...   
309  https://gnews.gg.go.kr/briefing/brief_gongbo_v...   

                                                 title       dept    reg_date  \
0                  아프리카돼지열병(ASF) 방역대책추진현황 200531 24시기준    동물방역위생과  2020.06.01   
1         경기북부 중심 하천인 신천 수질오염 심각… 친수공간 확보하고 하천생태계 복원해야      경기연구원  2020.06.01   
2    ‘대학생들이 구현해낸 자율주행 배달의 모습은?’  경기도 자율주행 배달 모빌리티 공...      

In [161]:
GG_LIST = db_select()

In [164]:
def save_excel(GG_LIST):
    excel = pd.ExcelWriter('경기도보도자료5월_1.xlsx')
    GG_LIST.to_excel(excel,'.',index=False)
    excel.save()

In [165]:
save_excel(GG_LIST)